# Llama-v2 7b for Retrieval Augmented Generation

In [9]:
%pip install langchain --user

Note: you may need to restart the kernel to use updated packages.


In [4]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.6 MB/s eta 0:00:0000:0100:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.74-cp311-cp311-linux_aarch64.whl size=622436 sha256=cedb7ff4b649622132974c1c8fcee40a50bf3b4eef7b56ac874d263a84eceacb
  Stored in directory: /home/jovyan/.cache/pip/wheels/3d/c9/d3/8515621dcf8921489f15fa822ae187ea2de185e92950119055
Successfully built llama-cpp-python


In [15]:
# Download a Llama.cpp optmized model
# List of models can be found at: https://huggingface.co/TheBloke
# In this case I will use Llama-2-7B-GGML: https://huggingface.co/TheBloke/Llama-2-7B-GGML
!mkdir -p /tmp/models/
!wget -P /tmp/models/ https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_0.bin

--2023-07-22 05:38:44--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_0.bin
Resolving huggingface.co (huggingface.co)... 65.8.178.118, 65.8.178.27, 65.8.178.93, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/8daa9615cce30c259a9555b1cc250d461d1bc69980a274b44d7eda0be78076d8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q4_0.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q4_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1690261102&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MDI2MTEwMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQxOWIzOTc1MmM4ZDBkY

In [16]:
from pathlib import Path

from langchain import LLMChain, PromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

model_dir = "/tmp/models/"
model_path = Path(model_dir)
model_file = list(model_path.glob("*.bin"))[0]

template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=str(model_file.resolve()),
    input={"temperature": 0.0, "max_length": 2000, "top_p": 1},
    callback_manager=callback_manager,
    verbose=True,
)

prompt = """
Question: Name all the planets in the solar system?
"""
llm(prompt)

llama.cpp: loading model from /tmp/models/llama-2-7b-chat.ggmlv3.q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 5185.72 MB (+ 1026.00 MB per state)
llama_new_context_with_model: kv self size  =  256.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 0 | WASM_


Answer: The eight planets in our solar system are:

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

Note: Pluto was previously considered a planet but is now classified as a dwarf planet by most astronomers.


llama_print_timings:        load time =  2933.96 ms
llama_print_timings:      sample time =    36.13 ms /    82 runs   (    0.44 ms per token,  2269.27 tokens per second)
llama_print_timings: prompt eval time =  4445.28 ms /    16 tokens (  277.83 ms per token,     3.60 tokens per second)
llama_print_timings:        eval time = 17599.56 ms /    81 runs   (  217.28 ms per token,     4.60 tokens per second)
llama_print_timings:       total time = 22276.60 ms


'\nAnswer: The eight planets in our solar system are:\n\n1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6. Saturn\n7. Uranus\n8. Neptune\n\nNote: Pluto was previously considered a planet but is now classified as a dwarf planet by most astronomers.'